# 1. Preparación de entorno


In [ ]:
# Instalacion de librerias
!pip install -q timm

# Imports
import os
import glob
import shutil
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
import timm
import torch.nn.functional as F


#2. HAM10000

In [ ]:
# Configuracion Drive
from google.colab import drive
drive.mount('/content/drive')

# carpeta de checkpoints
CHECKPOINT_DIR = "/content/drive/MyDrive/ucsp/implementacion_articulo/hao_checkpointsV3"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

if not os.path.exists('/content/data'):
    print("Descomprimiendo dataset...")
    !mkdir -p /content/data
    !unzip -q -o '/content/drive/MyDrive/HAM10000/skin-cancer-mnist-ham10000.zip' -d /content/data

# Unir HAM10000
img_dir = '/content/data/all_images'
os.makedirs(img_dir, exist_ok=True)

for part in ['HAM10000_images_part_1', 'HAM10000_images_part_2']:
    source_dir = os.path.join('/content/data', part)
    if os.path.exists(source_dir):
        for file_name in os.listdir(source_dir):
            shutil.move(os.path.join(source_dir, file_name), os.path.join(img_dir, file_name))

Mounted at /content/drive
Descomprimiendo dataset...


#3. Modulo de denoising

In [ ]:
'''
DenoisingModule: Implementación del modulo de Denoising
                 Encoder-Decoder sin pooling para preservar detalles espaciales
                 reducida para memoria
  ENTRADA:
    sin argumentos, define las capas internas
  SALIDA:
    Instancia del modelo nn.Module lista para forward
'''
class DenoisingModule(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = nn.Conv2d(3, 64, 3, padding=1)
        self.enc2 = nn.Conv2d(64, 64, 3, padding=1)
        # Decoder
        self.dec1 = nn.ConvTranspose2d(64, 64, 3, padding=1)
        self.dec2 = nn.ConvTranspose2d(64, 3, 3, padding=1)
        self.relu = nn.ReLU()

    '''
    forward: Ejecuta el paso de codificación y decodificación.
             skip connection para restaurar detalles perdidos.
      ENTRADA:
        x: tensor de imagen con ruido (Batch, 3, Alto, Ancho)
      SALIDA:
        out: tensor de imagen limpia (Batch, 3, Alto, Ancho)
    '''
    def forward(self, x):
        # Encoder
        e1 = self.relu(self.enc1(x))
        e2 = self.relu(self.enc2(e1))

        # Decoder + Skip connection
        # Se suma e1 (detalles de bajo nivel) a la salida de la primera deconvolucion
        d1 = self.relu(self.dec1(e2) + e1)
        out = self.dec2(d1)
        return out

#4. MS_CAM

In [ ]:
'''
MS_CAM: Implementación de Multi-Scale Channel Attention Module
        Calcula pesos de atención combinando contextos globales (con Pooling)
        y locales (con Convolucion punto a punto).
  ENTRADA:
    channels: entero, dimensión de canales del mapa de características
    r: entero, factor de reducción para el cuello de botella (default=4)
  SALIDA:
    Instancia del módulo lista para forward.
'''
class MS_CAM(nn.Module):
    def __init__(self, channels, r=4):
        super().__init__()
        inter_channels = int(channels // r)

        # Global Feature Attention
        # Usa Global Average Pooling seguido de PWConv
        self.global_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, inter_channels, kernel_size=1),
            nn.BatchNorm2d(inter_channels),
            nn.ReLU(),
            nn.Conv2d(inter_channels, channels, kernel_size=1),
            nn.BatchNorm2d(channels)
        )

        # Local Feature Attention
        # Usa solo PWConv sin pooling para preservar detalle espacial
        self.local_att = nn.Sequential(
            nn.Conv2d(channels, inter_channels, kernel_size=1),
            nn.BatchNorm2d(inter_channels),
            nn.ReLU(),
            nn.Conv2d(inter_channels, channels, kernel_size=1),
            nn.BatchNorm2d(channels)
        )

        self.sigmoid = nn.Sigmoid()

    '''
    forward: Genera los pesos de atención (alpha) sumando ramas global y local.
             M(X) = Sigmoid(L(X) + g(X)) según Ec. 5 del paper.
      ENTRADA:
        x: tensor de características (Batch, Channels, Alto, Ancho)
      SALIDA:
        w: tensor de pesos de atención (Batch, Channels, Alto, Ancho)
           con valores en rango [0, 1]
    '''
    def forward(self, x):
        g = self.global_att(x) # Rama global (Broadcasting de 1x1 a HxW)
        l = self.local_att(x)  # Rama local
        w = self.sigmoid(g + l)
        return w

#5. AFF

In [ ]:
'''
AFF: Submódulo de Fusión de Características Atencionales
     Fusiona características locales (ConvNeXt) y globales (Swin) mediante
     pesos dinámicos generados por el módulo MS-CAM.
  ENTRADA:
    channels: entero, número de canales de los mapas de características
    r: entero, factor de reducción para el módulo interno MS-CAM (default=4)
  SALIDA:
    Instancia del módulo AFF lista para forward.
'''
class AFF(nn.Module):
    def __init__(self, channels, r=4):
        super().__init__()
        self.mscam = MS_CAM(channels, r)

    '''
    forward: Ejecuta la fusión adaptativa de las entradas.
             Si las dimensiones espaciales son diferentes, interpola y para coincidir con x.
             Calcula Z = alpha * X + (1 - alpha) * Y.
      ENTRADA:
        x: tensor de características locales (ConvNeXt) -> (Batch, Canales, Alto, Ancho)
        y: tensor de características globales (Swin Transformer) -> (Batch, Canales, Alto, Ancho)
      SALIDA:
        z: tensor de características fusionadas -> (Batch, Canales, Alto, Ancho)
    '''
    def forward(self, x, y):
        # x: ConvNeXt features (Local)
        # y: Swin features (Global)

        # alineacion espacial: Asegurar que ambos tensores tengan el mismo tamaño (H, W)
        if x.size() != y.size():
            y = F.interpolate(y, size=x.shape[2:], mode='bilinear', align_corners=False)

        # fusion inicial para calcular el mapa de atencion
        # Se suman las características para que MS-CAM determine la importancia relativa
        initial_sum = x + y
        weights = self.mscam(initial_sum)

        # Fusion ponderada final (Ec. 3 del paper)
        # Z = alpha * X + (1 - alpha) * Y
        z = weights * x + (1 - weights) * y
        return z

#6. Modelo Híbrido

In [ ]:
'''
ConvNeXt_ST_AFF: Combina ConvNeXt y Swin Transformer
                 Tiene un módulo de Denoising inicial y fusión AFF en multiples etapas
  ENTRADA:
    num_classes: entero, número de clases para la capa de salida (default=7)
  SALIDA:
    Instancia del modelo nn.Module lista para entrenamiento/inferencia.
'''
class ConvNeXt_ST_AFF(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()

        # Módulo de Denoising (Pre-procesamiento dentro de la red)
        # Reduce artefactos antes de la extracción de características
        self.denoising = DenoisingModule()

        # Backbones
        # features_only=True para obtener mapas de características intermedios y no una clasificacion final directa.
        self.convnext = timm.create_model('convnext_tiny', pretrained=True, features_only=True)
        self.swin = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, features_only=True)

        # Obtener dimensiones de canales de cada stage para configurar AFF
        # ConvNeXt Tiny stages: [96, 192, 384, 768]
        c_dims = self.convnext.feature_info.channels()

        # Módulos de Fusión (AFF)
        # Se aplica fusión en las últimas 3 etapas principales para combinar representaciones de diferentes escalas.
        self.aff1 = AFF(c_dims[1])
        self.aff2 = AFF(c_dims[2])
        self.aff3 = AFF(c_dims[3])

        # Classification Head (Cabezal de Clasificación)
        # Procesa las características fusionadas finales para obtener logits
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.LayerNorm(c_dims[3]),
            nn.Linear(c_dims[3], num_classes)
        )

    '''
    forward: Define el flujo de datos a través de la red
             1. Denoising
             2. Extraccion (CNN + Transformer)
             3. Alineación de dimensiones
             4. Fusión progresiva de características
             5. Clasificacion
      ENTRADA:
        x: tensor de imágenes de entrada (Batch, 3, 224, 224)
      SALIDA:
        out: tensor de logits (Batch, num_classes)
    '''
    def forward(self, x):
        # 1. Denoising
        x_clean = self.denoising(x)

        # 2. Extracción de características (Backbones paralelos)
        c_feats = self.convnext(x_clean) # ConvNeXt devuelve formato (B, C, H, W)
        s_feats = self.swin(x_clean)     # Swin devuelve formato (B, H, W, C)

        # 3. Corrección de dimensiones para Swin Transformer
        # (B, H, W, C) -> (B, C, H, W) para coincidir con ConvNeXt
        s_feats = [s.permute(0, 3, 1, 2) for s in s_feats]

        # 4. fusion de caracteristicas (AFF)
        # Stage 1 Fusion (Feature map size ~ 28x28, Channels=192)
        f1 = self.aff1(c_feats[1], s_feats[1])

        # Stage 2 Fusion (Feature map size ~ 14x14, Channels=384)
        f2 = self.aff2(c_feats[2], s_feats[2])

        # Stage 3 Fusion (Feature map size ~ 7x7, Channels=768)
        # Se utilizan las características más profundas para la clasificación final
        f3 = self.aff3(c_feats[3], s_feats[3])

        # 5. Clasificación
        out = self.head(f3)
        return out


#7. Modelos individuales

In [ ]:
# obtener modelos individuales
def get_convnext_standalone(num_classes=7):
    model = timm.create_model('convnext_tiny', pretrained=True, num_classes=num_classes)
    return model

def get_swin_standalone(num_classes=7):
    model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=num_classes)
    return model

#8. Data augmentation

In [ ]:
# =========================
# DATASET HAM10000
# =========================
'''
HAM10000Dataset: Clase Dataset para cargar imagenes de HAM10000
                 Mapea las etiquetas de texto a índices numéricos y aplica transformaciones
  ENTRADA:
    df: DataFrame de pandas con metadatos (con 'image_id' y 'dx').
    img_dir: ruta al directorio que contiene las imágenes .jpg.
    transform: composición de transformaciones de torchvision a aplicar.
  SALIDA:
    Objeto Dataset iterable compatible con DataLoader.
'''
class HAM10000Dataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

        # Mapeo de clases a índices numéricos
        self.label_map = {
            'nv': 0, 'mel': 1, 'bkl': 2, 'bcc': 3,
            'akiec': 4, 'vasc': 5, 'df': 6
        }

    def __len__(self):
        return len(self.df)

    '''
    __getitem__: Carga una imagen y su etiqueta dado un índice.
      ENTRADA:
        idx: entero, índice del elemento a recuperar.
      SALIDA:
        Tupla (image, label) donde image es un tensor y label un tensor Long.
    '''
    def __getitem__(self, idx):
        # Construir ruta de imagen
        img_id = self.df.iloc[idx]['image_id']
        img_name = f"{img_id}.jpg"
        img_path = os.path.join(self.img_dir, img_name)

        # Cargar y convertir imagen
        image = Image.open(img_path).convert('RGB')

        # Obtener etiqueta numérica
        label_str = self.df.iloc[idx]['dx']
        label = self.label_map[label_str]

        # Aplicar transformaciones si existen
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

# =========================
# PREPARACIÓN DE DATALOADERS
# =========================
'''
get_dataloaders: Prepara los DataLoaders para Entrenamiento, Validación y Test.
                 Realiza el split 60/20/20 y configura las transformaciones según el escenario.
  ENTRADA:
    augment_scenario: string, define la estrategia de aumento (todos los conjuntos o solo entrenamiento).
    batch_size: entero, tamaño del batch para el entrenamiento (default=32).
  SALIDA:
    loaders: diccionario con claves ['train', 'val', 'test'] conteniendo los DataLoaders.
'''
def get_dataloaders(augment_scenario="train_only", batch_size=32):
    # Leer CSV de metadatos
    df = pd.read_csv('/content/data/HAM10000_metadata.csv')

    # --- SPLITTING 60/20/20 ---
    # Paso 1: Separar 20% para Test
    train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['dx'])

    # Del 80% restante, 25% para validacion
    # 25% de 80% = 20% del total. queda 60% para Training
    train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42, stratify=train_val_df['dx'])

    print(f"Split Sizes -> Train: {len(train_df)} (60%), Val: {len(val_df)} (20%), Test: {len(test_df)} (20%)")

    # --- TRANSFORMACIONES ---
    # Transformación con Aumentación (Rotación, Flip, Color, Crop)
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #imageNet
    ])

    # Transformación Base (Solo Resize y Normalización)
    base_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #imageNet
    ])

    # --- ESCENARIOS DE AUMENTACIÓN ---
    if augment_scenario == "augment_all":
        # Opc A: Aumentar TODO (Train, Val y Test)
        val_transform = train_transform
        test_transform = train_transform
        print("Aplicando Data Augmentation a Train, Val y Test")
    else:
        # Opc B: Aumentar solo Train
        val_transform = base_transform
        test_transform = base_transform
        print("Aplicando Data Augmentation SOLO a Train")

    # Crear Datasets
    img_folder = '/content/data/all_images'
    train_ds = HAM10000Dataset(train_df, img_folder, transform=train_transform)
    val_ds = HAM10000Dataset(val_df, img_folder, transform=val_transform)
    test_ds = HAM10000Dataset(test_df, img_folder, transform=test_transform)

    # Crear DataLoaders
    loaders = {
        'train': DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2),
        'val': DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2),
        'test': DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2)
    }

    return loaders

#9. Bucle de entrenamiento

In [ ]:
import torch.optim as optim
from tqdm import tqdm
import os
import torch.nn as nn
from sklearn.metrics import classification_report, accuracy_score

# =========================
# CONFIGURACIÓN DE DISPOSITIVO
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# =========================
# ENTRENAMIENTO
# =========================
'''
train_model: Ejecuta el bucle de entrenamiento y validación del modelo
             Utiliza optimizador SGD con Momentum y Cosine Annealing
             Guarda el checkpoint con mejor Accuracy de validacion
  ENTRADA:
    model: instancia de la red neuronal (nn.Module)
    loaders: diccionario con dataloaders ['train', 'val']
    model_name: string, nombre del modelo para el guardado de pesos
    scenario_name: string, identificador del escenario de aumento
    num_epochs: entero, numero de epocas a entrenar (default=20)
  SALIDA:
    model: la instancia del modelo con los pesos finales
'''
def train_model(model, loaders, model_name, scenario_name, num_epochs=20):
    criterion = nn.CrossEntropyLoss()

    # Configuración de Optimizador e Hiperparámetros del Paper:
    # SGD con momentum=0.9, weight_decay=0.0001
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

    # Scheduler: Reduce el Learning Rate siguiendo una curva coseno
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    model = model.to(device)
    best_acc = 0.0

    print(f"\nEntrenando {model_name} | Escenario: {scenario_name}")

    for epoch in range(num_epochs):
        # --- TRAIN (Entrenamiento) ---
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        # Bucle sobre lotes de entrenamiento con barra de progreso
        for images, labels in tqdm(loaders['train'], desc=f"Ep {epoch+1}/{num_epochs}", leave=False):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()           # Reiniciar gradientes
            outputs = model(images)         # Forward pass
            loss = criterion(outputs, labels)
            loss.backward()                 # Backward pass
            optimizer.step()                # Actualizar pesos

            # Métricas de entrenamiento
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = 100 * correct / total

        # --- VALIDATION (Validación) ---
        model.eval()
        val_correct = 0
        val_total = 0

        with torch.no_grad(): # Desactivar cálculo de gradientes para validación
            for images, labels in loaders['val']:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_acc = 100 * val_correct / val_total

        # Actualizar Learning Rate
        scheduler.step()

        print(f"Epoch {epoch+1}: Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # Guardar mejor modelo (Checkpoint)
        if val_acc > best_acc:
            best_acc = val_acc
            save_path = os.path.join(CHECKPOINT_DIR, f"{model_name}_{scenario_name}_best.pth")
            torch.save(model.state_dict(), save_path)
            print(f"Nuevo mejor modelo guardado en: {save_path}")

    print(f"Entrenamiento finalizado. Mejor Val Acc: {best_acc:.2f}%")
    return model

# =========================
# EVALUACIÓN (TEST)
# =========================
'''
evaluate_model: Realiza inferencia sobre el conjunto de Test y reporta métricas.
                Calcula Accuracy global y Reporte de Clasificación por clase.
  ENTRADA:
    model: instancia del modelo entrenado
    loader: DataLoader del conjunto de test
    model_name: string, nombre del modelo para el reporte
  SALIDA:
    No retorna valor (imprime resultados en consola).
'''
def evaluate_model(model, loader, model_name):
    model.eval()
    all_preds = []
    all_labels = []

    # Inferencia sin gradientes
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            # Acumular predicciones y etiquetas reales para sklearn
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Cálculo de métricas
    acc = accuracy_score(all_labels, all_preds)
    print(f"\nResultados de TEST para {model_name}:")
    print(f"Accuracy: {acc:.4f}")

    # Nombres de clases según HAM10000
    class_names = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']
    print(classification_report(all_labels, all_preds, target_names=class_names))

Usando dispositivo: cuda


#10. Entrenamiento y testing (modelos individuales)

In [ ]:
# Configuracion
NUM_EPOCHS = 100
SCENARIOS = ["augment_train_only", "augment_all"]

# Diccionario de constructores de modelos
models_to_run = {
    "ConvNeXt_Standalone": get_convnext_standalone,
    "Swin_Standalone": get_swin_standalone,
}

for scenario in SCENARIOS:
    print(f"\n{'='*40}")
    print(f"INICIANDO ESCENARIO DE AUMENTACIÓN: {scenario}")
    print(f"{'='*40}")

    # Obtener dataloaders especificos para este escenario
    loaders = get_dataloaders(augment_scenario=scenario, batch_size=32)

    for model_name, model_fn in models_to_run.items():
        # Instanciar modelo
        model = model_fn(num_classes=7)

        # Entrenar
        trained_model = train_model(model, loaders, model_name, scenario, num_epochs=NUM_EPOCHS)

        # Evaluar en Test
        print(f"Evaluando {model_name} en conjunto de TEST...")
        evaluate_model(trained_model, loaders['test'], model_name)

        # Liberar memoria GPU
        del model
        del trained_model
        torch.cuda.empty_cache()

print("\nexperimentos completados")


INICIANDO ESCENARIO DE AUMENTACIÓN: augment_train_only
Split Sizes -> Train: 6009 (60%), Val: 2003 (20%), Test: 2003 (20%)
Aplicando Data Augmentation SOLO a Train


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]


Entrenando ConvNeXt_Standalone | Escenario: augment_train_only


Epoch 1: Train Acc: 66.43% | Val Acc: 66.95%
Nuevo mejor modelo guardado en: /content/drive/MyDrive/ucsp/implementacion_articulo/hao_checkpointsV3/ConvNeXt_Standalone_augment_train_only_best.pth


...

Epoch 97: Train Acc: 99.75% | Val Acc: 88.47%


Epoch 98: Train Acc: 99.82% | Val Acc: 88.52%


Epoch 99: Train Acc: 99.77% | Val Acc: 88.52%


Epoch 100: Train Acc: 99.73% | Val Acc: 88.52%
Entrenamiento finalizado. Mejor Val Acc: 88.67%
Evaluando Swin_Standalone en conjunto de TEST...

Resultados de TEST para Swin_Standalone:
Accuracy: 0.8882
              precision    recall  f1-score   support

          nv       0.93      0.96      0.95      1341
         mel       0.74      0.67      0.71       223
         bkl       0.85      0.73      0.78       220
         bcc       0.83      0.87      0.85       103
       akiec       0.65      0.69      0.67        65
        vasc       0.92      0.86      0.89        28
          df       0.91      0.91      0.91        23

    accuracy                           0.89      2003
   macro avg       0.83      0.81      0.82      2003
weighted avg       0.89      0.89      0.89      2003


Entrenando ConvNeXt_ST_AFF_Hybrid | Escenario: augment_train_only


RuntimeError: The size of tensor a (192) must match the size of tensor b (28) at non-singleton dimension 1

#11.Entrenamiento y testing (híbrido)

In [ ]:
# Configuración
NUM_EPOCHS = 100 # Ajustar según tiempo disponible (Paper usa 60-100)
SCENARIOS = ["augment_train_only", "augment_all"]

# Diccionario de constructores de modelos
models_to_run = {
    "ConvNeXt_ST_AFF_Hybrid": ConvNeXt_ST_AFF
}

for scenario in SCENARIOS:
    print(f"\n{'='*40}")
    print(f"INICIANDO ESCENARIO DE AUMENTACIÓN: {scenario}")
    print(f"{'='*40}")

    # Obtener dataloaders específicos para este escenario
    loaders = get_dataloaders(augment_scenario=scenario, batch_size=32)

    for model_name, model_fn in models_to_run.items():
        # Instanciar modelo fresco
        model = model_fn(num_classes=7)

        # Entrenar
        trained_model = train_model(model, loaders, model_name, scenario, num_epochs=NUM_EPOCHS)

        # Evaluar en Test
        print(f"Evaluando {model_name} en conjunto de TEST...")
        evaluate_model(trained_model, loaders['test'], model_name)

        # Liberar memoria GPU
        del model
        del trained_model
        torch.cuda.empty_cache()

print("\nexperimentos completados")


INICIANDO ESCENARIO DE AUMENTACIÓN: augment_train_only
Split Sizes -> Train: 6009 (60%), Val: 2003 (20%), Test: 2003 (20%)
Aplicando Data Augmentation SOLO a Train


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]


Entrenando ConvNeXt_ST_AFF_Hybrid | Escenario: augment_train_only


...

Epoch 96: Train Acc: 99.57% | Val Acc: 86.37%


Epoch 97: Train Acc: 99.53% | Val Acc: 86.37%


Epoch 98: Train Acc: 99.58% | Val Acc: 86.57%


Epoch 99: Train Acc: 99.50% | Val Acc: 85.72%


Epoch 100: Train Acc: 99.55% | Val Acc: 86.72%
Entrenamiento finalizado. Mejor Val Acc: 87.12%
Evaluando ConvNeXt_ST_AFF_Hybrid en conjunto de TEST...

Resultados de TEST para ConvNeXt_ST_AFF_Hybrid:
Accuracy: 0.8802
              precision    recall  f1-score   support

          nv       0.93      0.96      0.95      1341
         mel       0.74      0.61      0.67       223
         bkl       0.76      0.73      0.74       220
         bcc       0.89      0.83      0.85       103
       akiec       0.64      0.65      0.64        65
        vasc       1.00      0.89      0.94        28
          df       0.81      0.91      0.86        23

    accuracy                           0.88      2003
   macro avg       0.82      0.80      0.81      2003
weighted avg       0.88      0.88      0.88      2003


experimentos completados
